In [ ]:
from google.colab import drive

import time
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error 

import torch, torch.nn as nn
import torch.nn.functional as F

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


#Cоздаём фичи для RNN 

In [ ]:
data_katago = pd.read_csv('./drive/MyDrive/dataset/data_katago.csv')
data_katago.head(2)

,W_rating,B_rating,W_nickname,B_nickname,Result,W_move,B_move,W_scoreLead,B_scoreLead,W_scoreSelfplay,B_scoreSelfplay,W_scoreStdev,B_scoreStdev,W_utility,B_utility,W_visits,B_visits,W_winrate,B_winrate
0,5d,5d,败亦喜418,海上游地,-1,pp dp nq pc oc nc lc kd rd re qf qh jb kb pb r...,qe cd qn pj od nd md ke je qd qc qb jc ib ic r...,-0.6840837 -0.573287904 -1.16207385 -1.3441191...,-0.195299417 -0.373908967 -0.209464177 -0.3537...,-1.03464818 -0.616376698 -1.72797191 -2.153860...,-0.401197761 -0.525847375 -0.39991954 -0.70452...,19.3432443 19.0153239 19.5488034 18.8073091 18...,18.9167735 18.7804246 18.6299643 18.9499844 18...,-0.106269752 -0.0630911261 -0.163935947 -0.213...,-0.0399894918 -0.0570140441 -0.0433106935 -0.0...,1 1 1 2 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 2 1 1 2 ...,1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 2 1 2 ...,0.448366866 0.469355524 0.420528233 0.39637158...,0.480593026 0.472265646 0.478934243 0.46121652...
1,2d,2d,王全洪18,zxh8209,1,dp dc fp eq ep er nq qq pr,pp pc fq gp fo fr hp lq pq,-0.293919653 -0.402171075 -0.499010742 -0.0830...,-0.195299417 -0.0381193496 -0.111136176 0.0382...,-0.505459726 -0.358326942 -0.563032448 0.01788...,-0.401197761 0.0693550929 -0.299834013 -0.1110...,19.1583855 19.0247085 19.452315 19.2031115 19....,18.9167735 18.7012602 19.1979015 18.852825 18....,-0.0368680026 -0.0490561791 -0.0672077993 -0.0...,-0.0399894918 -0.0245322315 -0.0429847252 -0.0...,1 1 1 2 1 2 1 1 1,1 1 1 1 1 1 2 1 2,0.482302636 0.475995645 0.467211381 0.49721161...,0.480593026 0.487631798 0.478944227 0.49210578...


In [ ]:
MAX_SIZE = 275

def pad_list(current_list, pad_size):
    new_list = np.pad(array=np.array(current_list, dtype='f'),
                                 pad_width=(0, pad_size), 
                                 mode='constant').tolist()
    return new_list


def convert_list_to_string(current_list):
    new_list = ', '.join(map(str, current_list))
    return new_list


def add_basic_stats(row, moves, pref=""):
    current_size = len(moves.score_delta)
    pad_size = MAX_SIZE - current_size

    row[pref + 'size'] = current_size
    row[pref + 'score'] = convert_list_to_string(pad_list(moves.score_delta,
                                                          pad_size))
    row[pref + 'winrate'] = convert_list_to_string(pad_list(moves.winrate_delta,
                                                          pad_size))
    row[pref + 'utility'] = convert_list_to_string(pad_list(moves.utility_delta,
                                                          pad_size))
    row[pref + 'selfplay'] = convert_list_to_string(pad_list(moves.selfplay_delta,
                                                          pad_size))
    row[pref + 'stddev'] = convert_list_to_string(pad_list(moves.stddev_delta,
                                                          pad_size))
                                 
  

def get_int_from_rank(rank):
    if rank[1] == 'k':
        return -int(rank[0]) + 1
    else:
        return int(rank[0])


def get_rank_from_int(x):
    if x > 0:
        return str(x) + "d"
    else:
        return str(-x + 1) + "k"


def add_meta(row):
    row['rank'] = get_int_from_rank(row['W_rating'])


def convert_to_lists(df):
    for i, row in tqdm(df.iterrows()):
        row['W_scoreLead'] = [float(x) for x in row['W_scoreLead'].split()]
        row['B_scoreLead'] = [float(x) for x in row['B_scoreLead'].split()]
        row['W_scoreSelfplay'] = [float(x) for x in row['W_scoreSelfplay'].split()]
        row['B_scoreSelfplay'] = [float(x) for x in row['B_scoreSelfplay'].split()]
        row['W_scoreStdev'] = [float(x) for x in row['W_scoreStdev'].split()]
        row['B_scoreStdev'] = [float(x) for x in row['B_scoreStdev'].split()]
        row['W_utility'] = [float(x) for x in row['W_utility'].split()]
        row['B_utility'] = [float(x) for x in row['B_utility'].split()]
        row['W_winrate'] = [float(x) for x in row['W_winrate'].split()]
        row['B_winrate'] = [float(x) for x in row['B_winrate'].split()]


class MovesInfo:
    def __init__(self, row, n_moves=None):
        moves_len = min(len(row['W_winrate']), len(row['B_winrate']))
        if n_moves is None:
            start_ind = 0
        else:
            start_ind = max(moves_len - n_moves - 1, 0)

        end_ind = moves_len
        print(len(row['W_winrate']), len(row['W_move']))

        self.winrate_delta = []
        self.score_delta = []
        self.utility_delta = []
        self.selfplay_delta = []
        self.stddev_delta = []
        self.cnt_moves = end_ind - start_ind
        self.move = row['W_move'].split()

        for i in range(start_ind, end_ind):
            self.winrate_delta.append(row['W_winrate'][i] - row['B_winrate'][i])
            self.score_delta.append(row['W_scoreLead'][i] - row['B_scoreLead'][i])
            self.utility_delta.append(row['W_utility'][i] - row['B_utility'][i])
            self.selfplay_delta.append(row['W_scoreSelfplay'][i] - row['B_scoreSelfplay'][i])
            self.stddev_delta.append(row['W_scoreStdev'][i] - row['B_scoreStdev'][i])


def add_all_game_stats(df):
    df['rank'] = None
    df['size'] = None
    df['score'] = None
    df['winrate'] = None
    df['utility'] = None
    df['selfplay'] = None
    df['stddev'] = None
    
    
    for i, row in tqdm(df.iterrows()):
        add_meta(row)
        add_basic_stats(row, MovesInfo(row))


def delete_non_scalar_parameters(df):
    df.drop(['W_rating', 'B_rating', 'W_move', 'B_move', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
             'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility', 'B_utility', 'W_visits', 'B_visits',
             'W_winrate', 'B_winrate', 'Result'], axis=1, inplace=True)


def get_feature_df(df):
    convert_to_lists(df)
    add_all_game_stats(df)
    delete_non_scalar_parameters(df)
    return df

In [ ]:
X = get_feature_df(data_katago.copy())

5it [00:00, 647.05it/s]
5it [00:00, 67.15it/s]

47 140
9 26
126 377
37 110
115 344


In [ ]:
X.head(2)

,W_nickname,B_nickname,rank,size,score,winrate,utility,selfplay,stddev
0,败亦喜418,海上游地,5,47,"-0.4887842833995819, -0.19937893748283386, -0....","-0.032226160168647766, -0.0029101220425218344,...","-0.06628026068210602, -0.006077081896364689, -...","-0.633450448513031, -0.09052932262420654, -1.3...","0.4264707863330841, 0.23489929735660553, 0.918..."
1,王全洪18,zxh8209,2,9,"-0.09862023591995239, -0.3640517294406891, -0....","0.001709609990939498, -0.011636152863502502, -...","0.003121489193290472, -0.02452394738793373, -0...","-0.10426196455955505, -0.4276820421218872, -0....","0.2416120022535324, 0.3234483003616333, 0.2544..."


In [ ]:
X.to_csv('./drive/MyDrive/dataset/game_sequence.csv', sep=';')

# RNN


In [ ]:
df = pd.read_csv('./drive/MyDrive/GO_dataset_after_script/dataset_100k_p1/game_sequence.csv', sep=';')
df.head(2)

,Unnamed: 0,W_nickname,B_nickname,rank,size,score,winrate,utility,selfplay,stddev
0,0,败亦喜418,海上游地,5,47,"-0.4887842833995819, -0.19937893748283386, -0....","-0.032226160168647766, -0.0029101220425218344,...","-0.06628026068210602, -0.006077081896364689, -...","-0.633450448513031, -0.09052932262420654, -1.3...","0.4264707863330841, 0.23489929735660553, 0.918..."
1,1,王全洪18,zxh8209,2,9,"-0.09862023591995239, -0.3640517294406891, -0....","0.001709609990939498, -0.011636152863502502, -...","0.003121489193290472, -0.02452394738793373, -0...","-0.10426196455955505, -0.4276820421218872, -0....","0.2416120022535324, 0.3234483003616333, 0.2544..."


In [ ]:
df['score'] = df['score'].apply(lambda x: np.array(list(map(float, x.split(','))), dtype='f'))
df['winrate'] = df['winrate'].apply(lambda x: np.array(list(map(float, x.split(','))), dtype='f'))
df['utility'] = df['utility'].apply(lambda x: np.array(list(map(float, x.split(','))), dtype='f'))
df['selfplay'] = df['selfplay'].apply(lambda x: np.array(list(map(float, x.split(','))), dtype='f'))
df['stddev'] = df['stddev'].apply(lambda x: np.array(list(map(float, x.split(','))), dtype='f'))
df.head(2)

,Unnamed: 0,W_nickname,B_nickname,rank,size,score,winrate,utility,selfplay,stddev
0,0,败亦喜418,海上游地,5,47,"[-0.48878428, -0.19937894, -0.95260966, -0.990...","[-0.03222616, -0.002910122, -0.05840601, -0.06...","[-0.06628026, -0.006077082, -0.12062525, -0.13...","[-0.63345045, -0.09052932, -1.3280524, -1.4493...","[0.4264708, 0.2348993, 0.9188391, -0.1426753, ..."
1,1,王全洪18,zxh8209,2,9,"[-0.098620236, -0.36405173, -0.38787457, -0.12...","[0.00170961, -0.011636153, -0.011732846, 0.005...","[0.0031214892, -0.024523947, -0.024223074, 0.0...","[-0.104261965, -0.42768204, -0.26319844, 0.128...","[0.241612, 0.3234483, 0.2544135, 0.3502865, 0...."


In [ ]:
features_and_size = ['score', 'winrate', 'utility', 'selfplay', 'stddev', 'size']
features = ['score', 'winrate', 'utility', 'selfplay', 'stddev']
MOVES = 275
RANK_SHIFT = 2 
test_prob = 0.2
train_size = int(len(df) * (1 - test_prob))
test_size = int(len(df) * test_prob)

In [ ]:
X = df[features_and_size]
y = df['rank'] + RANK_SHIFT
X_pretrain, X_pretest, y_pretrain, y_pretest = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_sizes = X_pretrain['size'].to_numpy()
X_test_sizes = X_pretest['size'].to_numpy()
y_train = y_pretrain.to_numpy()
y_test = y_pretest.to_numpy()

In [ ]:
def convert_row_to_nparray(row):
    return np.concatenate(row[features].to_numpy()).reshape((MOVES, -1))

def convert_df_to_nparray(dataframe, size):
    column_arrays = []
    for index, row in tqdm(dataframe.iterrows()):
        column_arrays.append(convert_row_to_nparray(row))
    
    return np.concatenate(column_arrays).reshape((size, MOVES, -1))

In [ ]:
X_train = convert_df_to_nparray(X_pretrain, train_size)
X_test = convert_df_to_nparray(X_pretest, test_size)

80000it [00:56, 1406.61it/s]
20000it [00:14, 1405.17it/s]


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
n_input = len(X_train[0][0]) # 5 features per move
n_hidden = 32 # Hidden layer num of features
n_classes = 10 # 10 dans and kus 

In [ ]:
class CharRNNLoop(nn.Module):
    def __init__(self, input_size=n_input, hidden_layers=n_hidden, 
                 num_layers=2, output_size= n_classes, emb_size=27):
        super(self.__class__, self).__init__()
        # self.lin = nn.Linear(input_size, emb_size)

        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_layers,
                           num_layers=num_layers, batch_first=True)
        self.relu = nn.LeakyReLU()
        self.classifier = nn.Linear(hidden_layers, 1)
        # self.smax = nn.LogSoftmax()
        
    def forward(self, x):
        # out, _ = self.rnn(self.lin(x))
        out, _ = self.rnn(x)
        out_unp, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        answer = out_unp[:, -1, :]
        answer = self.relu(answer)
        answer = self.classifier(answer)
        # answer = self.smax(answer)
        return answer

In [ ]:
model = CharRNNLoop().to(device)
model.load_state_dict(torch.load('./drive/MyDrive/modelLSTM'))
opt = torch.optim.Adam(model.parameters(), lr=4e-4)
loss_func = F.mse_loss
epochs = 200
batch_size = 8192

In [ ]:
def train_model(model, loss_fn, opt, n_epochs: int):
    min_mae = 100.0
    train_loss = []
    test_loss = []
    test_mae = []
    
    for epoch in range(epochs):
        permutation_train = torch.randperm(train_size)
        permutation_test = torch.randperm(test_size)

        ep_train_loss = []
        ep_test_loss = []
        ep_test_mae = []
        start_time = time.time()
        
        model.train(True)
        
        for i in range(0, train_size, batch_size):
            opt.zero_grad()
            idx = permutation_train[i: min(i + batch_size, train_size)]
            size = len(idx)

            X_batch = torch.FloatTensor(X_train[idx]).to(device)
            X_size = torch.LongTensor(X_train_sizes[idx])
            X_pack = nn.utils.rnn.pack_padded_sequence(X_batch, X_size,
                                                       batch_first=True,
                                                       enforce_sorted=False)
            y_batch = torch.FloatTensor(y_train[idx]).reshape((size, 1)).to(device)
            predictions = model.forward(X_pack)        
            loss = loss_fn(predictions, y_batch)
            loss.backward()
            opt.step()
            ep_train_loss.append(loss.item())
        
        model.train(False)
        with torch.no_grad():
            for i in range(0, test_size, batch_size):
                idx = permutation_test[i: min(i + batch_size, test_size)]
                size = len(idx)

                X_batch = torch.FloatTensor(X_test[idx]).to(device) 
                X_size = torch.LongTensor(X_test_sizes[idx])
                X_pack = nn.utils.rnn.pack_padded_sequence(X_batch, X_size, 
                                                           batch_first=True,
                                                           enforce_sorted=False)
                y_batch = torch.FloatTensor(y_test[idx]).reshape((size, 1)).to(device)
                predictions = model.forward(X_pack)
                loss = loss_fn(predictions, y_batch)

                ep_test_loss.append(loss.item())
                y_pred = predictions
                ep_test_mae.append(mean_absolute_error(y_pred.cpu(), y_batch.cpu()))
                

        # print the results for this epoch:
        print(f'Epoch {epoch + 1} of {n_epochs} took {time.time() - start_time:.3f}s')

        train_loss.append(np.mean(ep_train_loss))
        test_loss.append(np.mean(ep_test_loss))
        if (np.mean(ep_test_mae) < min_mae):
            torch.save(model.state_dict(), 'modelLSTM')
            min_mae = np.mean(ep_test_mae)

        test_mae.append(np.mean(ep_test_mae))

        print(f"\t  training loss: {train_loss[-1]:.6f}")
        print(f"\t  test loss: {test_loss[-1]:.6f}")
        print(f"\t  test mae: {test_mae[-1]:.3f}")
        print(f"\t  min mae: {min_mae:.3f}")
    return train_loss, test_loss, test_mae, min_mae

In [ ]:
train_loss, test_loss, test_mae, min_mae = train_model(model, loss_fn=loss_func, opt=opt, n_epochs=epochs)

Epoch 1 of 200 took 11.956s
	  training loss: 8.517941
	  test loss: 8.683942
	  test mae: 2.529
	  min mae: 2.529
Epoch 2 of 200 took 12.042s
	  training loss: 8.517568
	  test loss: 8.690575
	  test mae: 2.533
	  min mae: 2.529
Epoch 3 of 200 took 11.981s
	  training loss: 8.510195
	  test loss: 8.706726
	  test mae: 2.535
	  min mae: 2.529
Epoch 4 of 200 took 11.998s
	  training loss: 8.505086
	  test loss: 8.669210
	  test mae: 2.527
	  min mae: 2.527
Epoch 5 of 200 took 11.929s
	  training loss: 8.502510
	  test loss: 8.684370
	  test mae: 2.529
	  min mae: 2.527
Epoch 6 of 200 took 11.878s
	  training loss: 8.498464
	  test loss: 8.704104
	  test mae: 2.533
	  min mae: 2.527
Epoch 7 of 200 took 11.943s
	  training loss: 8.495107
	  test loss: 8.642829
	  test mae: 2.522
	  min mae: 2.522
Epoch 8 of 200 took 11.887s
	  training loss: 8.489844
	  test loss: 8.670351
	  test mae: 2.529
	  min mae: 2.522
Epoch 9 of 200 took 12.004s
	  training loss: 8.484717
	  test loss: 8.714597
	 

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/modelLSTM')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import pylab
import warnings as w
import os
import torch
import torch, torch.nn as nn

def plot_train_process(train_loss, test_loss, test_accuracy):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].set_title('Loss')
    axes[0].plot(train_loss, label='train')
    axes[0].plot(test_loss, label='test')
    axes[0].legend()

    axes[1].set_title('Test accuracy')
    axes[1].plot(test_accuracy)